In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [6]:
women = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.18890814558058924


In [7]:
mean_age = train_data["Age"].mean()

In [8]:
def filterdata(data):
    # change rows with missing age to mean_age
    data.loc[data[data["Age"].isna()].index, "Age"] = mean_age
    
    # filter x
    filters = data.loc[:,["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Embarked"]]

    # get the sex column : 1 if male, 0 if female
    sex = ( filters["Sex"] == "male").astype(int)

    # get the embarked column : 0 if nan, 1 if C, 2 if Q and 3 if S
    Embarked_values = [np.nan, "C", "Q", "S"]
    embarked = [Embarked_values.index(emb) for emb in filters["Embarked"]]
    
    # get tickets number
    ticket = [ (tick.split(" ")[-1]) for tick in filters["Ticket"]]

    filters["Sex"] = sex
    filters["Embarked"] = embarked
    filters["Ticket"] = ticket
    filters.drop(filters[filters["Ticket"] == 'LINE'].index, inplace = True)
    filters["Ticket"] = filters["Ticket"].astype(float)
    
    # filter x
    x = filters.loc[:,["Pclass", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Embarked"]]
    x = x.to_numpy()
    x = keras.utils.normalize(x, axis = 0)
    
    # fliter y
    y = filters["Survived"]
    y = y.to_numpy()
    

    return x, y

In [9]:
X,Y = filterdata(train_data)
m,n = X.shape
print(X.shape)
print(Y.shape)

(887, 8)
(887,)


In [10]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

model = keras.Sequential([layers.Dense(units = 64, input_shape = (m,n), activation="relu")])
model.add(layers.Dense(units = 64, activation="relu"))
model.add(layers.Dense(units = 32, activation="relu"))
model.add(layers.Dense(units = 32, activation="relu"))
model.add(layers.Dense(units = 1, activation="sigmoid"))

adam = keras.optimizers.Adam(learning_rate = lr_schedule)
model.compile(optimizer=adam, loss="BinaryCrossentropy")
model.fit(X, Y, epochs=500)

Epoch 1/500
28/28 [==============================] - 1s 1ms/step - loss: 0.6443
Epoch 2/500
28/28 [==============================] - 0s 2ms/step - loss: 0.4842
Epoch 3/500
28/28 [==============================] - 0s 1ms/step - loss: 0.5352
Epoch 4/500
28/28 [==============================] - 0s 1ms/step - loss: 0.4883
Epoch 5/500
28/28 [==============================] - 0s 1ms/step - loss: 0.4840
Epoch 6/500
28/28 [==============================] - 0s 1ms/step - loss: 0.5166
Epoch 7/500
28/28 [==============================] - 0s 1ms/step - loss: 0.4615
Epoch 8/500
28/28 [==============================] - 0s 1ms/step - loss: 0.4518
Epoch 9/500
28/28 [==============================] - 0s 1ms/step - loss: 0.4835
Epoch 10/500
28/28 [==============================] - 0s 1ms/step - loss: 0.5244
Epoch 11/500
28/28 [==============================] - 0s 1ms/step - loss: 0.4456
Epoch 12/500
28/28 [==============================] - 0s 1ms/step - loss: 0.4792
Epoch 13/500
28/28 [=================

In [11]:
count = 0
y_hat = (model.predict(X) > 0.5).astype(int)
for i in range(m):
    if(y_hat[i] == Y[i]):
        count += 1
print(count/m)

0.8692220969560316


In [12]:
mean_age = train_data["Age"].mean()
mean_fare = train_data["Fare"].mean()

In [13]:
def filtertestdata(data):
    # replace missing age with mean age
    data.loc[data[data["Age"].isna()].index, "Age"] = mean_age
    data.loc[data[data["Fare"].isna()].index, "Fare"] = mean_fare
    
    # filter x
    filters = data.loc[:,["Pclass", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Embarked"]]

    # get the sex column 1 if male, 0 if female
    sex = ( filters["Sex"] == "male").astype(int)

    # get the embarked column 0 if nan, 1 if C, 2 if Q and 3 if S
    Embarked_values = [np.nan, "C", "Q", "S"]
    embarked = [Embarked_values.index(emb) for emb in filters["Embarked"]]
    
    # get tickets number
    ticket = [ (tick.split(" ")[-1]) for tick in filters["Ticket"]]

    filters["Sex"] = sex
    filters["Embarked"] = embarked
    filters["Ticket"] = ticket
    filters.drop(filters[filters["Ticket"] == 'LINE'].index, inplace = True)
    filters["Ticket"] = filters["Ticket"].astype(float)
    
    # filter x
    x = filters.loc[:,["Pclass", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Embarked"]]
    x = x.to_numpy()
    x = keras.utils.normalize(x, axis = 0)
    

    return x

In [14]:
test_filtred = test_data.copy()
test_filtred = filtertestdata(test_filtred)

In [15]:
predictions = (model.predict(test_filtred) > 0.5).astype(int)

predictions = predictions.reshape(418,)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('lrdecay_v88_submission.csv', index=False)